In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d


from transformer_IDAO import TransformerIDAO
from clustering.clustering import Clustering2D
from prediction import Model2D
from validation.IDAO_validator_2d_model import IDAO_validator_2d_model
import importlib

%matplotlib inline 

In [2]:
#full_test = pd.read_csv("./data/test.csv", index_col='id')
#full_train = pd.read_csv("./data/train.csv", index_col='id')
full_test = pd.read_csv("./data/test.csv", index_col='id')
full_train = pd.read_csv("./data/train.csv", index_col='id')

transformer = TransformerIDAO()

idao_2d_splitter = IDAO_validator_2d_model(5, 5)

In [3]:
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    return np.mean(np.abs((satellite_predicted_values - satellite_true_values) 
        / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))

In [5]:
## список из значений smape (на тесте) для каждого из sat_id
smape_per_sat_id = []
## номер фолда для вывода логов
n_folds = 0

smape_list = []
## пока проверяю предсказания в 2D!s
target_columns = ['x_pred', 'y_pred', 'Vx_pred', 'Vy_pred']#, 'Vz_pred']
true_columns = ['x_transformed', 'y_transformed', 'Vx_transformed', 'Vy_transformed']#, 'Vz']

for n_clusters in range(1, 200):
    for trn_idxs, tst_idxs in idao_2d_splitter.split(full_train[:50000]):
        train_df = None
        test_df = None
        for sat_id in full_train[:50000]['sat_id'].unique():
            if train_df is None:
                train_df = full_train.iloc[trn_idxs[sat_id, 0]:trn_idxs[sat_id, 1]].copy()
            else:
                train_df = pd.concat([train_df, full_train.iloc[trn_idxs[sat_id, 0]:trn_idxs[sat_id, 1]]], axis=0).copy()
            if test_df is None:
                test_df = full_train.iloc[tst_idxs[sat_id, 0]:tst_idxs[sat_id, 1]].copy()
            else:
                test_df = pd.concat([test_df, full_train.iloc[tst_idxs[sat_id, 0]:tst_idxs[sat_id, 1]]], axis=0).copy()

        transformer.fit_transform(train_df, train_coefs_V=None)

        clustering1 = Clustering2D('AgglomerativeClustering', n_clusters)
        clusters_map = clustering1.fit_predict(transformer.train_coefs)

        transformer.fit_transform(test_df, train=False)

        model = Model2D(10)

        test_df, train_df = model.predict(train_df.copy(), test_df.copy(), clusters_map)

        test_df['z_pred'] = 0.0
        test_df['Vz_pred'] = 0.0
        ## так как проверяю предсказания в  2d
    #     test_df = transformer.inv_transform(test_df)

        n_folds += 1
    #     print(test_df)
        preds = test_df[target_columns].values
        preds1 = test_df[true_columns].values



        print(f'fold: {n_folds}, target_col: {target_columns},\
                      smape: {1 - smape(test_df[target_columns].values, test_df[true_columns].values):.5f}')
        smape_list.append(1 - smape(test_df[target_columns].values, test_df[true_columns].values))

/home/paniquex/Documents/idao_oski/transformer_IDAO.py:35: RuntimeWarning: invalid value encountered in true_divide
  a /= norm
/home/paniquex/Documents/idao_oski/transformer_IDAO.py:36: RuntimeWarning: invalid value encountered in true_divide
  b /= norm
/home/paniquex/Documents/idao_oski/transformer_IDAO.py:37: RuntimeWarning: invalid value encountered in true_divide
  c /= norm


156 0
4 0
2239 2000
2229 2000
5888 5828
5903 5828
6632 6620
6670 6620
7316 7294
7312 7294
10067 10055
10253 10055
16379 16308
16353 16308
17140 17104
17130 17104
18039 17822
17840 17822
22513 22453
22456 22453
23429 23194
23204 23194
26822 26606
26625 26606
30362 30326
30329 30326
30901 30889
30893 30889
33053 32897
33045 32897
34796 34780
34793 34780
40002 39990
40016 39990
40492 40360
40362 40360
41920 41805
41815 41805
44653 44641
44645 44641
45574 45562
45563 45562
46064 45932
45934 45932
47279 47243
47294 47243
47760 47748
47750 47748
48317 48221
48243 48221
49182 49170
49172 49170
50610 50597
50599 50597
51043 50839
51032 50839
52897 52886
52911 52886
53763 53751
53824 53751
54649 54638
54858 54638
56879 56866
56869 56866
57533 57473
57475 57473
58280 58052
58271 58052
60305 60293
60297 60293
62355 62342
62395 62342
63231 63219
63440 63219
68150 68141
68357 68141
72245 72089
72093 72089
73716 73608
73706 73608
74863 74647
74667 74647
76984 76979
77214 76979
80564 80551
80701 8055

/home/paniquex/Documents/idao_oski/transformer_IDAO.py:35: RuntimeWarning: invalid value encountered in true_divide
  a /= norm
/home/paniquex/Documents/idao_oski/transformer_IDAO.py:36: RuntimeWarning: invalid value encountered in true_divide
  b /= norm
/home/paniquex/Documents/idao_oski/transformer_IDAO.py:37: RuntimeWarning: invalid value encountered in true_divide
  c /= norm


180 143
148 143
2648 2422
2638 2422
5888 5828
5951 5828
6632 6620
6694 6620
7629 7536
7770 7536
10770 10761
10764 10761
16451 16308
16472 16308
17140 17104
17179 17104
18360 18338
18571 18338
22561 22453
22456 22453
23598 23547
23780 23547
27231 27015
27033 27015
30435 30326
30329 30326
31093 31072
31158 31072
33197 32980
33213 32980
35600 35364
35447 35364
40002 39990
40016 39990
40637 40410
40410 40410
42113 42110
42128 42110
44653 44641
44645 44641
45636 45562
45585 45562
46185 45968
45982 45968
47328 47243
47344 47243
47760 47748
47750 47748
48317 48221
48243 48221
49230 49230
49244 49230
50644 50597
50619 50597
51211 51029
51104 51029
52898 52886
52888 52886
53763 53751
53753 53751
54889 54866
55098 54866
56878 56866
56869 56866
57581 57473
57475 57473
58328 58284
58368 58284
60497 60483
60489 60483
62351 62342
62443 62342
63759 63746
63968 63746
68586 68577
68771 68577
72390 72169
72189 72169
73812 73608
73730 73608
75103 74865
74883 74865
77393 77371
77599 77371
80660 80637
8070

/home/paniquex/Documents/idao_oski/transformer_IDAO.py:35: RuntimeWarning: invalid value encountered in true_divide
  a /= norm
/home/paniquex/Documents/idao_oski/transformer_IDAO.py:36: RuntimeWarning: invalid value encountered in true_divide
  b /= norm
/home/paniquex/Documents/idao_oski/transformer_IDAO.py:37: RuntimeWarning: invalid value encountered in true_divide
  c /= norm


...

565 334
341 334
3079 2843
3070 2843
6057 5838
6072 5838
6632 6620
6694 6620
7797 7778
8011 7778
11694 11467
11496 11467
16547 16318
16543 16318
17237 17104
17227 17104
18862 18853
19073 18853
22633 22453
22456 22453
23910 23900
24116 23900
27641 27423
27442 27423
30435 30326
30329 30326
31286 31283
31302 31283
33365 33142
33381 33142
35949 35949
35970 35949
40002 39990
40016 39990
40781 40556
40579 40556
42594 42416
42416 42416
44701 44692
44693 44692
45575 45562
45662 45562
46328 46106
46126 46106
47378 47243
47368 47243
47893 47748
47770 47748
48317 48280
48291 48280
49399 49381
49389 49381
50668 50597
50619 50597
51259 51245
51248 51245
52921 52919
52935 52919
53835 53790
53801 53790
55105 55100
55338 55100
57021 56866
56869 56866
57654 57473
57475 57473
58521 58520
58536 58520
60713 60699
60705 60699
62348 62342
62342 62342
64273 64272
64291 64272
69025 69013
69211 69013
72558 72329
72333 72329
73932 73695
73922 73695
75319 75094
75099 75094
77993 77763
77984 77763
80804 80801
8101

/home/paniquex/Documents/idao_oski/transformer_IDAO.py:35: RuntimeWarning: invalid value encountered in true_divide
  a /= norm
/home/paniquex/Documents/idao_oski/transformer_IDAO.py:36: RuntimeWarning: invalid value encountered in true_divide
  b /= norm
/home/paniquex/Documents/idao_oski/transformer_IDAO.py:37: RuntimeWarning: invalid value encountered in true_divide
  c /= norm


589 526
533 526
3487 3265
3502 3265
6081 5921
6144 5921
6728 6663
6694 6663
8038 8020
8252 8020
12186 12173
12180 12173
16618 16402
16567 16402
17237 17104
17276 17104
19390 19369
19600 19369
22705 22477
22623 22477
24271 24253
24477 24253
28051 27832
27972 27832
30557 30326
30329 30326
31551 31495
31687 31495
33533 33304
33525 33304
36752 36533
36623 36533
40002 39990
40016 39990
40925 40701
40723 40701
42738 42721
42729 42721
45025 44789
44925 44789
45574 45562
45712 45562
46472 46244
46462 46244
47378 47243
47441 47243
47917 47748
47770 47748
48389 48380
48411 48380
49567 49531
49533 49531
50692 50597
50619 50597
51668 51460
51464 51460
53231 53010
53030 53010
53883 53883
53897 53883
55345 55334
55554 55334
57116 56881
56892 56881
57702 57477
57571 57477
58762 58756
58777 58756
60929 60914
60921 60914
62372 62350
62367 62350
64801 64799
64819 64799
69465 69449
69676 69449
72726 72489
72502 72489
74028 73804
73994 73804
75559 75323
75339 75323
78161 78155
78176 78155
80972 80964
8118

/home/paniquex/Documents/idao_oski/transformer_IDAO.py:35: RuntimeWarning: invalid value encountered in true_divide
  a /= norm
/home/paniquex/Documents/idao_oski/transformer_IDAO.py:36: RuntimeWarning: invalid value encountered in true_divide
  b /= norm
/home/paniquex/Documents/idao_oski/transformer_IDAO.py:37: RuntimeWarning: invalid value encountered in true_divide
  c /= norm


ValueError: Cannot extract more clusters than samples: 50 clusters where given for a tree with 49 leaves.

In [ ]:
plt.plot(preds1[:, 0], label='true x')
plt.plot(preds[:, 0], label='pred x')
plt.legend()

In [ ]:
test_df[true_columns]

In [ ]:
transformer.fit_transform(full_train)
print(full_train.columns)

In [ ]:
clustering1 = Clustering2D('AgglomerativeClustering', 600)
clusters_map = clustering1.fit_predict(transformer.train_coefs)

In [ ]:
model = Model2D(WIDTH=30)
new_test, new_train = model.predict(full_train.copy(), full_test.copy(), clusters_map)

In [ ]:
# new_test['z_pred'] = 0.0
# new_test['Vz_pred'] = 0.0
# new_test = transformer.inv_transform(new_test)

In [ ]:
full_test = new_test

In [ ]:
# отрисовка нужного рядя для анализа
import numpy as np
import pandas as pd
import lightgbm as lgbm
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import time
from sklearn.linear_model import LinearRegression
from tqdm import tqdm_notebook as tqdm


figure = plt.figure(figsize=(10, 5))


sat_id = 3
real_col_name = 'x_transformed'
sim_col_name = real_col_name + '_sim'
pred_col_name = real_col_name + '_pred'

fig = go.Figure()
fig.add_trace(go.Scatter(x=full_test[full_test.sat_id == sat_id].epoch,
                             y=full_test[full_test.sat_id == sat_id]['x_pred'],
                             name="Estimated " + real_col_name,
                             ))

# fig.add_trace(go.Scatter(x=full_test[full_test.sat_id == sat_id].epoch,
#                          y=full_test[full_test.sat_id == sat_id][sim_col_name],
#                          name="Simulation " + real_col_name,
#                          ))

# fig.add_trace(go.Scatter(x=full_test[full_test.sat_id == sat_id].epoch,
#                          y=full_test[full_test.sat_id == sat_id][sim_col_name],
#                          name="Sim_test " + real_col_name,
#                          ))
fig.add_trace(go.Scatter(x=full_train[full_train.sat_id == sat_id].epoch,
                         y=full_train[full_train.sat_id == sat_id][real_col_name],
                         name="Real " + real_col_name,
                         ))

# fig.add_trace(go.Scatter(x=full_train[full_train.sat_id == sat_id].epoch,
#                          y=full_train[full_train.sat_id == sat_id][sim_col_name],
#                          name="Simulation " + real_col_name,
#                          ))

fig.update_layout(title_text='Time Series for ' + real_col_name,
                  xaxis_rangeslider_visible=True,
                  yaxis_title=real_col_name)
fig.show()

In [ ]:
full_test